In [1]:
import pandas as pd
import numpy as np
import os

## Clean Population Estimate Data and Demographic Growth Estimate Data

In [2]:
# Load in population estimate data and clean
df = pd.read_csv('population_estimates.csv')
df.rename(columns={"Age group 3 6": "Age Group"}, inplace= True)
df['Age Group'].ffill(inplace=True)
df.loc[df["Gender"] == "Men+", "Gender"] = "Men"
df.loc[df["Gender"] == "Women+", "Gender"] = "Women"

# Unpivot the provinces
df = df.melt(id_vars=['Gender', 'Age Group', 'Reference period'], var_name = "Province", value_name= "Population")

# Change data type to int
df["Population"] = df["Population"].str.replace(",","").astype(int)

# Create new age group for 1 to 4 years
df_grouped = (
df[df['Age Group'].isin(['1 year', '2 years', '3 years', '4 years'])]
.groupby(['Gender', 'Reference period', 'Province'], as_index=False)['Population'].sum()
)

# Rename the column to reflect the new grouping
df_grouped["Age Group"] = "1 to 4 years"

# Delete rows from population estimate dataframe and append df_grouped to final_df
df = df[(df['Age Group'] != '1 year') &
(df['Age Group'] != '2 years') &
(df['Age Group'] != '3 years') &
(df['Age Group'] != '4 years')]
df.reset_index(drop=True, inplace=True)

df = pd.concat([df_grouped, df], ignore_index=True)
df

C:\Users\retar\AppData\Local\Temp\ipykernel_19156\2694922176.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age Group'].ffill(inplace=True)


,Gender,Reference period,Province,Population,Age Group
0,Men,2014,Alberta,109325,1 to 4 years
1,Men,2014,British Columbia,92425,1 to 4 years
2,Men,2014,Manitoba,33319,1 to 4 years
3,Men,2014,New Brunswick,14378,1 to 4 years
4,Men,2014,Newfoundland and Labrador,9787,1 to 4 years
...,...,...,...,...,...
5195,Women,2019,Nunavut,8,90 years and older
5196,Women,2020,Nunavut,8,90 years and older
5197,Women,2021,Nunavut,14,90 years and older
5198,Women,2022,Nunavut,14,90 years and older


## Load and clean Mortality Rate Data

In [3]:
# Load in mortality rate data and clean
mortality_df = pd.read_csv('mortality_rate.csv')

# Forward fill blanks
mortality_df['Province'].ffill(inplace=True)
mortality_df['Age Group'].ffill(inplace=True)
mortality_df['Sex'].ffill(inplace=True)

# Rename values/columns for consistency between dataframes
mortality_df.rename(columns={"Sex": "Gender"}, inplace= True)
mortality_df.loc[mortality_df["Gender"] == "Males", "Gender"] = "Men"
mortality_df.loc[mortality_df["Gender"] == "Females", "Gender"] = "Women"
mortality_df.loc[mortality_df["Age Group"] == "90 years and over", "Age Group"] = "90 years and older"
mortality_df.loc[mortality_df["Province"] == 'Northwest Territories 15', 'Province'] = 'Northwest Territories'

# Change Mortality Rate to int
mortality_df.loc[mortality_df["Mortality rate per 1,000 population"] == "..", "Mortality rate per 1,000 population"] = np.nan
mortality_df["Mortality rate per 1,000 population"] = mortality_df["Mortality rate per 1,000 population"].astype(float)

mortality_df

C:\Users\retar\AppData\Local\Temp\ipykernel_19156\3285436874.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mortality_df['Province'].ffill(inplace=True)
C:\Users\retar\AppData\Local\Temp\ipykernel_19156\3285436874.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

,Province,Age Group,Gender,Reference period,"Mortality rate per 1,000 population"
0,Newfoundland and Labrador,0 years,Men,2014,8.8
1,Newfoundland and Labrador,0 years,Men,2015,5.3
2,Newfoundland and Labrador,0 years,Men,2016,6.0
3,Newfoundland and Labrador,0 years,Men,2017,4.4
4,Newfoundland and Labrador,0 years,Men,2018,6.9
...,...,...,...,...,...
5195,Nunavut,90 years and older,Women,2019,0.0
5196,Nunavut,90 years and older,Women,2020,272.7
5197,Nunavut,90 years and older,Women,2021,300.0
5198,Nunavut,90 years and older,Women,2022,142.9


## Load and clean Fertility Rate Data

In [4]:
# Load in fertility rate data and clean
fertility_df = pd.read_csv('fertility_rates.csv')
fertility_df.rename(columns={"Geography 7": "Province"}, inplace= True)
fertility_df['Province'].ffill(inplace=True)
fertility_df['Gender'].ffill(inplace=True)

# Unpivot the Age Group and create Fertility Rate Column
fertility_df = fertility_df.melt(id_vars=['Gender', 'Province', 'Reference period'], var_name = "Age Group", value_name= "Fertility Rate")
fertility_df.loc[fertility_df["Fertility Rate"] == "..", "Fertility Rate"] = np.nan
fertility_df["Fertility Rate"] = fertility_df["Fertility Rate"].astype(float)

# Rename values/columns for consistency between dataframes
fertility_df.loc[fertility_df["Gender"] == "Females", "Gender"] = "Women"
fertility_df.loc[fertility_df["Province"] == 'Northwest Territories 12', 'Province'] = 'Northwest Territories'

fertility_df['Province'].unique()
fertility_df

C:\Users\retar\AppData\Local\Temp\ipykernel_19156\953709997.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fertility_df['Province'].ffill(inplace=True)
C:\Users\retar\AppData\Local\Temp\ipykernel_19156\953709997.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,Gender,Province,Reference period,Age Group,Fertility Rate
0,Women,Newfoundland and Labrador,2014,15 to 19 years,15.9
1,Women,Newfoundland and Labrador,2015,15 to 19 years,11.2
2,Women,Newfoundland and Labrador,2016,15 to 19 years,12.7
3,Women,Newfoundland and Labrador,2017,15 to 19 years,11.1
4,Women,Newfoundland and Labrador,2018,15 to 19 years,10.0
...,...,...,...,...,...
905,Women,Nunavut,2019,45 to 49 years,0.0
906,Women,Nunavut,2020,45 to 49 years,2.9
907,Women,Nunavut,2021,45 to 49 years,1.0
908,Women,Nunavut,2022,45 to 49 years,1.0


## Load and clean Net Migration Data

In [5]:
# Load in the demographic growth estimate data and clean
migration_df = pd.read_csv('net_migration.csv')
migration_df.rename(columns={"Geography": "Province"}, inplace= True)
migration_df.rename(columns={"Age group": "Age Group"}, inplace= True)

migration_df['Province'].ffill(inplace=True)
migration_df['Gender'].ffill(inplace=True)
migration_df['Age Group'].ffill(inplace=True)

migration_df.loc[migration_df["Reference period"] == "2014 / 2015", "Reference period"] = 2014
migration_df.loc[migration_df["Reference period"] == "2015 / 2016", "Reference period"] = 2015
migration_df.loc[migration_df["Reference period"] == "2016 / 2017", "Reference period"] = 2016
migration_df.loc[migration_df["Reference period"] == "2017 / 2018", "Reference period"] = 2017
migration_df.loc[migration_df["Reference period"] == "2018 / 2019", "Reference period"] = 2018
migration_df.loc[migration_df["Reference period"] == "2019 / 2020", "Reference period"] = 2019
migration_df.loc[migration_df["Reference period"] == "2020 / 2021", "Reference period"] = 2020
migration_df.loc[migration_df["Reference period"] == "2021 / 2022", "Reference period"] = 2021
migration_df.loc[migration_df["Reference period"] == "2022 / 2023", "Reference period"] = 2022
migration_df.loc[migration_df["Reference period"] == "2023 / 2024", "Reference period"] = 2023

# Change data type to int
migration_df["Net-migration"] = migration_df["Net-migration"].str.replace(",","").astype(int)

# Create new age group for 1 to 4 years
df_grouped = (
migration_df[migration_df['Age Group'].isin(['1 year', '2 years', '3 years', '4 years'])]
.groupby(['Gender', 'Reference period', 'Province'], as_index=False)['Net-migration'].sum()
)

# Rename the column to reflect the new grouping
df_grouped["Age Group"] = "1 to 4 years"

# Delete rows from population estimate dataframe and append df_grouped to final_df
migration_df = migration_df[(migration_df['Age Group'] != '1 year') &
                (migration_df['Age Group'] != '2 years') &
                (migration_df['Age Group'] != '3 years') &
                (migration_df['Age Group'] != '4 years')]

migration_df.reset_index(drop=True, inplace=True)

migration_df = pd.concat([df_grouped, migration_df], ignore_index=True)
migration_df

C:\Users\retar\AppData\Local\Temp\ipykernel_19156\284232426.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  migration_df['Province'].ffill(inplace=True)
C:\Users\retar\AppData\Local\Temp\ipykernel_19156\284232426.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,Gender,Reference period,Province,Net-migration,Age Group
0,Men,2014,Alberta,113,1 to 4 years
1,Men,2014,British Columbia,623,1 to 4 years
2,Men,2014,Manitoba,-214,1 to 4 years
3,Men,2014,New Brunswick,-29,1 to 4 years
4,Men,2014,Newfoundland and Labrador,108,1 to 4 years
...,...,...,...,...,...
5195,Women,2019,Nunavut,0,90 years and older
5196,Women,2020,Nunavut,0,90 years and older
5197,Women,2021,Nunavut,0,90 years and older
5198,Women,2022,Nunavut,0,90 years and older


## Join the datasets together

In [6]:
df1 = pd.merge(left=df, right=mortality_df, how='left', left_on=['Gender', 'Province', 'Reference period', 'Age Group'], right_on= ['Gender', 'Province', 'Reference period', 'Age Group'])
df2 = pd.merge(left=df1,right=fertility_df, how='left',left_on=['Gender', 'Province', 'Reference period', 'Age Group'], right_on=['Gender', 'Province', 'Reference period', 'Age Group'])
df_final = pd.merge(left=df2,right=migration_df, how='left',left_on=['Gender', 'Province','Reference period', 'Age Group'], right_on=['Gender', 'Province', 'Reference period', 'Age Group'])

# Add a migration rate column (per 1,000 population)
df_final['Migration Rate per 1,000'] = (df_final['Net-migration'] / df_final['Population']) * 1000
df_final

,Gender,Reference period,Province,Population,Age Group,"Mortality rate per 1,000 population",Fertility Rate,Net-migration,"Migration Rate per 1,000"
0,Men,2014,Alberta,109325,1 to 4 years,0.2,NaN,113,1.033615
1,Men,2014,British Columbia,92425,1 to 4 years,0.1,NaN,623,6.740600
2,Men,2014,Manitoba,33319,1 to 4 years,0.3,NaN,-214,-6.422762
3,Men,2014,New Brunswick,14378,1 to 4 years,0.1,NaN,-29,-2.016970
4,Men,2014,Newfoundland and Labrador,9787,1 to 4 years,0.1,NaN,108,11.035046
...,...,...,...,...,...,...,...,...,...
5195,Women,2019,Nunavut,8,90 years and older,0.0,NaN,0,0.000000
5196,Women,2020,Nunavut,8,90 years and older,272.7,NaN,0,0.000000
5197,Women,2021,Nunavut,14,90 years and older,300.0,NaN,0,0.000000
5198,Women,2022,Nunavut,14,90 years and older,142.9,NaN,0,0.000000


## Calculate Average Rates for Mortality, Fertility, and Migration

In [7]:
# Calculate average rates for fertility, mortality, and migration
average_rates = (df_final[df_final["Reference period"] <= 2023].groupby(["Gender", "Province", "Age Group"])
                 [["Mortality rate per 1,000 population", "Fertility Rate", "Migration Rate per 1,000"]].mean().reset_index())

# Create dictionaries for average rates
mortality_dict = average_rates.set_index(["Gender", "Province", "Age Group"])["Mortality rate per 1,000 population"].to_dict()
fertility_dict = average_rates.set_index(["Gender", "Province", "Age Group"])["Fertility Rate"].to_dict()
migration_dict = average_rates.set_index(["Gender", "Province", "Age Group"])["Migration Rate per 1,000"].to_dict()

## Create Population Projections for 2024 to 2033

In [64]:
# Initialize the projected dataset
df_projected = df_final[df_final["Reference period"] == 2023].copy()

# Age group transition mapping (define order of transitions)
age_groups_order = ["0 years", "1-4 years", "5-9 years", "10-14 years", "15-19 years",
                    "20-24 years", "25-29 years", "30-34 years", "35-39 years",
                    "40-44 years", "45-49 years", "50-54 years", "55-59 years",
                    "60-64 years", "65-69 years", "70-74 years", "75-79 years",
                    "80-84 years", "85-89 years", "90 years and over"]

# Iterate through projection years
projection_years = range(2024, 2034)

for year in projection_years:
    # Copy the latest year’s data and update the reference period
    current_year = df_projected[df_projected["Reference period"] == year - 1].copy()
    current_year["Reference period"] = year

    # Calculate last year's population dynamically
    def get_last_year_population(row):
        return df_projected.loc[(df_projected["Gender"] == row["Gender"]) &
                                (df_projected["Age Group"] == row["Age Group"]) &
                                (df_projected["Province"] == row["Province"]) &
                                (df_projected["Reference period"] == year - 1), "Population"].values[0]  # Retrieve the single value

    current_year["Population"] = current_year.apply(get_last_year_population, axis=1)

    # Fetch corresponding rates
    mortality_rate = current_year.apply(lambda row: mortality_dict.get((row["Gender"], row["Province"], row["Age Group"]), 0), axis=1)
    fertility_rate = current_year.apply(lambda row: fertility_dict.get((row["Gender"], row["Province"], row["Age Group"]), 0), axis=1)
    migration_rate = current_year.apply(lambda row: migration_dict.get((row["Gender"], row["Province"], row["Age Group"]), 0), axis=1)
    
    # Calculate births, deaths, and migration effects and replace NaN with 0
    births = ((current_year["Population"] * fertility_rate / 1000)/2).fillna(0).astype(int)
    deaths = (current_year["Population"] * mortality_rate / 1000).fillna(0).astype(int)
    migration_effect = (current_year["Population"] * migration_rate / 1000).fillna(0).astype(int)

    # Initialize new columns for deaths and migration effect
    current_year["Births"] = 0
    current_year["Deaths"] = deaths
    current_year["Migration Effect"] = migration_effect

    # Handle births (only for 0 years age group)
    for province in current_year["Province"].unique():
        # Calculate total births for men and women in the current province
        total_births = births[current_year["Province"] == province].sum()

        # Update the "Births" column for "0 years" in the current_year DataFrame
        current_year.loc[(current_year["Gender"] == "Men") & (current_year["Age Group"] == "0 years") &
        (current_year["Province"] == province), "Births"] = total_births

        current_year.loc[(current_year["Gender"] == "Women") & (current_year["Age Group"] == "0 years") &
        (current_year["Province"] == province), "Births"] = total_births

    # Handle age transitions
    next_year_population = current_year.copy()

    for i in range(len(age_groups_order) - 1):
        age_group = age_groups_order[i]
        next_age_group = age_groups_order[i + 1]

        for gender in ["Men", "Women"]:
            for province in current_year["Province"].unique():
                # Get the total population of the current age group
                current_pop = current_year.loc[(current_year["Gender"] == gender) &
                                                (current_year["Age Group"] == age_group) &
                                                (current_year["Province"] == province), "Population"]
                
                # If the age group spans multiple years, transition a fraction of it
                if "-" in age_group:
                    age_span = int(age_group.split("-")[1].split()[0]) - int(age_group.split("-")[0]) + 1 # Calculate span
                elif age_group == "0 years": # Handle "0 Years" transition
                    age_span = 1
                elif age_group == "90 years and over":
                    continue
                else:
                    raise ValueError(f"Unexpected age group format: {age_group}")  # Catch errors early

                # Compute transition population
                transition_pop = (current_pop / age_span).fillna(0).astype(int)

                # Subtract transitioning population from the current age group
                next_year_population.loc[(next_year_population["Gender"] == gender) &
                                        (next_year_population["Age Group"] == age_group) &
                                        (next_year_population["Province"] == province), "Population"] -= transition_pop

                # Add transitioning population to the next age group
                next_year_population.loc[(next_year_population["Gender"] == gender) & 
                                         (next_year_population["Age Group"] == next_age_group) & 
                                         (next_year_population["Province"] == province), "Population"] += transition_pop

    # Apply final adjustments for births, deaths, and migration
    next_year_population["Population"] += next_year_population["Births"] - next_year_population["Deaths"] + next_year_population["Migration Effect"]

    # Append this year’s data to the projected dataset
    df_projected = pd.concat([df_projected, next_year_population], ignore_index=True)

    # Update population
    #current_year["Population"] += current_year["Births"] - deaths + migration_effect

    # Append this year’s data to the projected dataset
    #df_projected = pd.concat([df_projected, current_year], ignore_index=True)

df_projected.drop(['Mortality rate per 1,000 population', 'Fertility Rate', 
                'Migration Rate per 1,000', 'Net-migration'], axis=1, inplace=True)

df_projected.drop(df_projected[df_projected['Reference period'] == 2023].index, inplace = True)
df_projected

,Gender,Reference period,Province,Population,Age Group,Births,Deaths,Migration Effect
520,Men,2024,Alberta,107275,1 to 4 years,0.0,22.0,67.0
521,Men,2024,British Columbia,91796,1 to 4 years,0.0,12.0,346.0
522,Men,2024,Manitoba,33845,1 to 4 years,0.0,11.0,-193.0
523,Men,2024,New Brunswick,14414,1 to 4 years,0.0,1.0,103.0
524,Men,2024,Newfoundland and Labrador,8148,1 to 4 years,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...
5715,Women,2033,Nunavut,179,70 to 74 years,0.0,6.0,-2.0
5716,Women,2033,Nunavut,64,75 to 79 years,0.0,5.0,0.0
5717,Women,2033,Nunavut,23,80 to 84 years,0.0,2.0,0.0
5718,Women,2033,Nunavut,9,85 to 89 years,0.0,1.0,0.0


## Append Projected Years to Historical Years

In [65]:
df_complete = pd.concat([df_final, df_projected], ignore_index=True)

df_complete.drop(['Mortality rate per 1,000 population', 'Fertility Rate', 
                'Migration Rate per 1,000', 'Net-migration'], axis=1, inplace=True)

df_complete.to_csv('Canada_Population_Projection.csv', sep='\t', encoding='utf-8', index=False, header=True)